<a href="https://colab.research.google.com/github/shoreey/Advanced-Classification-System-for-High-Risk-Colon-Polyps-in-Confocal-Microendoscopy-Images/blob/main/Polyp_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import io
import pandas as pd
import random
import scipy.io as sio
from scipy import ndimage
import scipy
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision import models

import torch
import torch.nn as nn
import math
from functools import partial
from torch.autograd import Variable

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import time
import datetime
import copy

from PIL import Image

In [ ]:
#read excel file
BASE_DIR='.\MI_Data'
data_df = pd.read_excel(".\MI_polyps_clear.xlsx")
data_df.head()
#print (data_df)

num_classes = 1

In [ ]:
class Polyps_Dataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms

    def __getitem__(self,index):
        filename=self.df.iloc[index]['fp'];

        im_path=os.path.join(self.imfolder,filename.strip())
        data = sio.loadmat(im_path)
        xstack=data['xstack']
        sz=xstack.shape
        #print(sz)
        #x = torch.from_numpy(data['xstack'])
        if(len(sz)<3):
            x=torch.zeros([1,3,224,224])
            tmp=Image.fromarray(xstack).convert('RGB')
            if(self.transforms):
                tmp=self.transforms(tmp)
            x[0,:,:,:]=tmp.type(torch.FloatTensor);
        else:
            x=torch.zeros([sz[2],3,224,224])

            for ct in range(sz[2]):
                tmp=Image.fromarray(xstack[:,:,ct]).convert('RGB')
                if(self.transforms):
                    tmp=self.transforms(tmp)
                x[ct,:,:,:]=tmp.type(torch.FloatTensor);

        if(self.train):
            y=self.df.iloc[index]['Class2']
            return x,y
        else:
            return x

    def __len__(self):
        return len(self.df)

In [ ]:
train, valid = train_test_split(
    data_df,
    test_size=0.2,
    random_state=42,
    stratify=data_df.Class2
)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def my_collate(batch):
    data=torch.tensor([])
    target=torch.tensor([])
    pat=torch.tensor([])

    for index, item in enumerate(batch):
        sz=item[0].size()
        data=torch.cat((data,item[0]),dim=0)

        lab=torch.ones(sz[0],)*item[1].astype(int)
        target=torch.cat((target,lab),dim=0)

        inst=torch.ones(sz[0],)*index
        pat=torch.cat((pat,inst),dim=0)

    data=data.unsqueeze(1)
    return [data, target, pat]

# reset index on both dataframes
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train_targets = train.Class2.values

# targets for validation
valid_targets = valid.Class2.values

train_dataset=Polyps_Dataset(
    df=train,
    imfolder=BASE_DIR,
    train=True,
    transforms=train_transform
)

valid_dataset=Polyps_Dataset(
    df=valid,
    imfolder=BASE_DIR,
    train=True,
    transforms=valid_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    #num_workers=4,
    shuffle=True,
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    #num_workers=4,
    shuffle=False,
)

In [ ]:
!pip install efficientnet_pytorch

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

# Freezing the entire model
for param in model.parameters():
    param.requires_grad = False

# Replacing the last layer and unfreezing it
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes)


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16464 sha256=4f72b375478e49690891a1df62c127b5a6e10b360a0a6ada789c600ff0e534dc
  Stored in directory: c:\users\shaur\appdata\local\pip\cache\wheels\9c\3f\43\e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\shaur/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:01<00:00, 16.7MB/s]


Loaded pretrained weights for efficientnet-b0


NameError: name 'num_classes' is not defined

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)


In [ ]:
def train_model(datasets, dataloaders, model, criterion, optimizer, scheduler, num_epochs, device, class_weights):

    N_iter_internal = 10
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100.0
    best_acc=0.0


    for epoch in range(num_epochs):
        preds=np.empty((0,1))
        targets=np.empty((0,1))

        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corr = 0.0
            running_corrects = 0.0
            running_corrects_0=0.0
            running_corrects_1=0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels=labels.type(torch.FloatTensor).to(device)

                # Zero out the grads
                optimizer.zero_grad()

                # Forward
                # Track history in train mode
                with torch.set_grad_enabled(phase == 'train'):
                    model=model.to(device)
                    outputs = model(inputs.squeeze(0))

                    ###############################################################################
                    #Multiclass MIL
                    #score=torch.ones((4,)).to(device)
                    #score[3]=1-torch.exp(-torch.sum(p[:,3],dim=0))
                    #score[2]=(1-score[3])*(1-torch.exp(-torch.sum(p[:,2],dim=0)))
                    #score[1]=(1-score[3])*(1-score[2])*(1-torch.exp(-torch.sum(p[:,1],dim=0)))
                    #score[0]=(1-score[3])*(1-score[2])*(1-score[1])*(1-torch.exp(-torch.sum(p[:,0],dim=0)))
                    #pred=torch.argmax(score)
                    ###############################################################################


                    #loss=criterion(score.unsqueeze(0),labels-1)
                    #loss=criterion(score,labels)

                    #p=torch.nn.functional.softmax(outputs,dim=1)
                    p=torch.sigmoid(outputs)
                    #score=torch.zeros(1,2).to(device)
                    #score[0,1]=1-torch.exp(-torch.sum(p[:,1],dim=0))
                    #score[0,0]=torch.exp(-torch.sum(p[:,1],dim=0))
                    score=1-torch.exp(-torch.sum(p,dim=0))
                    loss=criterion(score,labels)
                    if labels==0:
                        loss=class_weight[0]*criterion(score,labels)
                    else:
                        loss=class_weight[1]*criterion(score,labels)

                    pred=torch.argmax(score)
                    #batch_score=MIL_aggregator(outputs,1)
                    #loss = criterion(batch_score.unsqueeze(0).to(device), (scores/15).type(torch.FloatTensor).to(device))

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(pred==(labels))
                if(labels==0 and score<=0.5):#[0]>batch_score[1]):
                    running_corrects_0 += 1
                if(labels==1 and score>0.5):#[1]>batch_score[0]):
                    running_corrects_1 += 1

                #running_corrects_0 += torch.sum(batch_score[labels==0] == labels.data[labels.data==0])
                #running_corrects_1 += torch.sum(batch_score[labels.data==1] == labels.data[labels.data==1])

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/len(datasets[phase])
            epoch_acc = (running_corrects)/(len(datasets[phase]))
            acc_0=running_corrects_0/(sum(datasets[phase].df['Class2']==0))
            acc_1=running_corrects_1/(sum(datasets[phase].df['Class2']==1))
            epoch_balacc = 0.5*(acc_0+acc_1)

            print('{} Loss: {:.4f} Acc: {:.4f} Balanced Acc: {:.4f}  Acc 0: {:.4f}  Acc 1: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_balacc,acc_0,acc_1))

            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'Polyps_ResNet_binaryclass_MIL.pth')


            #if phase == 'valid':
            #    plt.plot(targets,preds,'o')
            #    plt.show()

            #if phase == 'valid' and epoch_loss < best_loss:
            #    best_loss = epoch_loss
            #    best_model_wts = copy.deepcopy(model.state_dict())

            print()

    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model


In [ ]:
class_sample_count = np.array([len(np.where(train_targets == t)[0]) for t in np.unique(train_targets)])
weight = 1. / class_sample_count
class_weight=torch.from_numpy(weight)
class_weight=class_weight/class_weight.max()
class_weight=class_weight.type(torch.FloatTensor)
print(class_weight)
print(class_sample_count)

tensor([1.0000, 0.3661])
[ 41 112]


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device=torch.device('cpu')

datasets={'train':train_dataset,'valid':valid_dataset}
dataloaders={'train':train_loader,'valid':valid_loader}

#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.0001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

#class_weight[0]=1
#criterion=nn.L1Loss()
#criterion=nn.MSELoss()
criterion=nn.BCELoss()
#criterion=nn.MSELoss()
#criterion=nn.CrossEntropyLoss(weight=class_weight.to(device))

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.1)

# Decay LR by a factor of 0.1 every 7 epoch
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

num_epochs=100

trained_model=train_model(datasets,dataloaders,model,criterion,optimizer,scheduler,num_epochs,device, class_weight)


Epoch 0/99
----------
train Loss: 0.4387 Acc: 0.2680 Balanced Acc: 0.4546  Acc 0: 0.5610  Acc 1: 0.3482

valid Loss: 0.3753 Acc: 0.2821 Balanced Acc: 0.5049  Acc 0: 0.5455  Acc 1: 0.4643

Epoch 1/99
----------
train Loss: 0.4352 Acc: 0.2680 Balanced Acc: 0.5058  Acc 0: 0.6098  Acc 1: 0.4018

valid Loss: 0.4139 Acc: 0.2821 Balanced Acc: 0.4545  Acc 0: 0.9091  Acc 1: 0.0000

Epoch 2/99
----------
train Loss: 0.4171 Acc: 0.2680 Balanced Acc: 0.4932  Acc 0: 0.6829  Acc 1: 0.3036

valid Loss: 0.3973 Acc: 0.2821 Balanced Acc: 0.5195  Acc 0: 0.1818  Acc 1: 0.8571

Epoch 3/99
----------
train Loss: 0.4356 Acc: 0.2680 Balanced Acc: 0.4594  Acc 0: 0.4634  Acc 1: 0.4554

valid Loss: 0.3767 Acc: 0.2821 Balanced Acc: 0.5406  Acc 0: 0.5455  Acc 1: 0.5357

Epoch 4/99
----------
train Loss: 0.4338 Acc: 0.2680 Balanced Acc: 0.4501  Acc 0: 0.5610  Acc 1: 0.3393

valid Loss: 0.3789 Acc: 0.2821 Balanced Acc: 0.5844  Acc 0: 0.4545  Acc 1: 0.7143

Epoch 5/99
----------
train Loss: 0.4320 Acc: 0.2680 Balance

valid Loss: 0.3958 Acc: 0.2821 Balanced Acc: 0.5617  Acc 0: 0.9091  Acc 1: 0.2143

Epoch 44/99
----------
train Loss: 0.4327 Acc: 0.2680 Balanced Acc: 0.5066  Acc 0: 0.6829  Acc 1: 0.3304

valid Loss: 0.4022 Acc: 0.2821 Balanced Acc: 0.5438  Acc 0: 0.9091  Acc 1: 0.1786

Epoch 45/99
----------
train Loss: 0.4223 Acc: 0.2680 Balanced Acc: 0.4968  Acc 0: 0.6098  Acc 1: 0.3839

valid Loss: 0.4132 Acc: 0.2821 Balanced Acc: 0.5081  Acc 0: 0.9091  Acc 1: 0.1071

Epoch 46/99
----------
train Loss: 0.4317 Acc: 0.2680 Balanced Acc: 0.4924  Acc 0: 0.6098  Acc 1: 0.3750

valid Loss: 0.3973 Acc: 0.2821 Balanced Acc: 0.5244  Acc 0: 0.7273  Acc 1: 0.3214

Epoch 47/99
----------
train Loss: 0.4198 Acc: 0.2680 Balanced Acc: 0.4826  Acc 0: 0.5366  Acc 1: 0.4286

valid Loss: 0.3950 Acc: 0.2821 Balanced Acc: 0.6299  Acc 0: 0.5455  Acc 1: 0.7143

Epoch 48/99
----------
train Loss: 0.4194 Acc: 0.2680 Balanced Acc: 0.5605  Acc 0: 0.5854  Acc 1: 0.5357

valid Loss: 0.4507 Acc: 0.2821 Balanced Acc: 0.4545  Ac

train Loss: 0.4065 Acc: 0.2680 Balanced Acc: 0.5420  Acc 0: 0.7805  Acc 1: 0.3036

valid Loss: 0.3969 Acc: 0.2821 Balanced Acc: 0.5049  Acc 0: 0.5455  Acc 1: 0.4643

Epoch 88/99
----------
train Loss: 0.4079 Acc: 0.2680 Balanced Acc: 0.5019  Acc 0: 0.7805  Acc 1: 0.2232

valid Loss: 0.3945 Acc: 0.2821 Balanced Acc: 0.5422  Acc 0: 0.7273  Acc 1: 0.3571

Epoch 89/99
----------
train Loss: 0.4086 Acc: 0.2680 Balanced Acc: 0.5462  Acc 0: 0.8780  Acc 1: 0.2143

valid Loss: 0.3993 Acc: 0.2821 Balanced Acc: 0.5325  Acc 0: 0.6364  Acc 1: 0.4286

Epoch 90/99
----------
train Loss: 0.4040 Acc: 0.2680 Balanced Acc: 0.5372  Acc 0: 0.8780  Acc 1: 0.1964

valid Loss: 0.3934 Acc: 0.2821 Balanced Acc: 0.5049  Acc 0: 0.5455  Acc 1: 0.4643

Epoch 91/99
----------
train Loss: 0.4079 Acc: 0.2680 Balanced Acc: 0.5632  Acc 0: 0.8049  Acc 1: 0.3214

valid Loss: 0.3985 Acc: 0.2821 Balanced Acc: 0.5601  Acc 0: 0.7273  Acc 1: 0.3929

Epoch 92/99
----------
train Loss: 0.4077 Acc: 0.2680 Balanced Acc: 0.5396  Ac

In [ ]:
device

In [ ]:
model=model.to(device)
inputs, labels = next(iter(train_loader))
inputs = inputs.to(device)
labels=labels.type(torch.LongTensor).to(device)
outputs=model(inputs.squeeze(0))

p=torch.nn.functional.sigmoid(outputs)
score=torch.ones((4,)).to(device)
score[3]=1-torch.exp(-torch.sum(p[:,3],dim=0))
score[2]=(1-score[3])*(1-torch.exp(-torch.sum(p[:,2],dim=0)))
score[1]=(1-score[3])*(1-score[2])*(1-torch.exp(-torch.sum(p[:,1],dim=0)))
score[0]=(1-score[3])*(1-score[2])*(1-score[1])*(1-torch.exp(-torch.sum(p[:,0],dim=0)))

pred=torch.argmax(score)

loss=criterion(score.unsqueeze(0),labels-1)
loss.backward()
optimizer.step()
#score=1-torch.exp(-torch.sum(p[:,2]))
#loss=criterion(labels,score)

In [ ]:
model=model.to(device)
inputs, labels = next(iter(train_loader))
inputs = inputs.to(device)
labels=labels.type(torch.LongTensor).to(device)
outputs=model(inputs.squeeze(0))

p=torch.nn.functional.softmax(outputs,dim=1)
score=torch.zeros(1,2).to(device)
score[0,1]=1-torch.exp(-torch.sum(p[:,1],dim=0))
score[0,0]=-torch.exp(-torch.sum(p[:,1],dim=0))


RuntimeError: expand(torch.cuda.FloatTensor{[2]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [ ]:
score

tensor([[0., 0.]], device='cuda:0')

In [ ]:


model=model.to(device)
inputs = inputs.to(device)
labels=labels.type(torch.LongTensor).to(device)
outputs = model(inputs.squeeze(0))

p=torch.nn.functional.sigmoid(outputs)
score=1-torch.exp(-torch.sum(p,dim=0))
#p=torch.nn.functional.softmax(outputs,dim=1)
#score=torch.zeros(1,2).to(device)
#score[0,1]=1-torch.exp(-torch.sum(p,dim=0))
#score[0,0]=-torch.exp(-torch.sum(p[:,1],dim=0))
criterion=nn.BCELoss()
loss=criterion(score,labels.type(torch.FloatTensor).to(device))


#criterion=nn.L1Loss()
#loss=criterion(score,labels)
print(outputs)
print(score)
print(labels)
print(loss)

In [ ]:
mean_loss=0;
count=0;
batch_score=torch.zeros((10,1))
batch_labels=torch.zeros((10,1))

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.0001)

count=0
num_epochs=50
for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)

    running_loss=0
    for inputs, labels in (train_loader):

        if(count==10):
            count=0
            batch_score=torch.zeros((10,1))
            batch_labels=torch.zeros((10,1))


        model=model.to(device)
        inputs = inputs.to(device)
        labels=labels.type(torch.LongTensor).to(device)
        outputs = model(inputs.squeeze(0))

        p=torch.nn.functional.sigmoid(outputs)
        score=1-torch.exp(-torch.sum(p,dim=0))
        #p=torch.nn.functional.softmax(outputs,dim=1)
        #score=torch.zeros(1,2).to(device)
        #score[0,1]=1-torch.exp(-torch.sum(p,dim=0))
        #score[0,0]=-torch.exp(-torch.sum(p[:,1],dim=0))

        batch_score[count]=score
        batch_labels[count]=labels.type(torch.FloatTensor)
        #obs_loss=criterion(score,labels.type(torch.FloatTensor).to(device))
        #if(labels==1):
        #    loss=loss+class_weight[1]*obs_loss
        #else:
        #    loss=loss+class_weight[0]*obs_loss

        #mean_loss=mean_loss+loss.item()
        count=count+1
        if (count==9):
            loss=criterion(batch_score, batch_labels)
            running_loss += loss.item()*10
            loss.backward()
            optimizer.step()
    epoch_loss = running_loss/len(train_dataset)
    print('{} Loss: {:.4f} '.format('Train', epoch_loss))
    #print(mean_loss/count)

In [ ]:
data_label=[]
data_estimate=[]
device = torch.device("cpu")

for inputs, labels in (train_loader):

    model=model.to(device)
    inputs = inputs.to(device)
    labels=labels.type(torch.LongTensor).to(device)
    outputs = model(inputs.squeeze(0))

    p=torch.nn.functional.sigmoid(outputs)
    score=1-torch.exp(-torch.sum(p,dim=0))
    data_estimate.append(score)
    data_label.append(labels)

In [ ]:
error=0
for ct in range(165):
    error=error+((data_estimate[ct]>0.5)==data_label[ct])


In [ ]:
data_estimate

In [ ]:
score